# Week 09 - BioPython and Final Remarks

<div class="topics">
    <div style="padding-left: 15px;">
        This lecture will cover:
        <ul>
        <li>BioPython</li>
        <ul>
            <li>An Introduction to the BioPython module</li>
            <li>Ways of making life easier in computational biology</li>
        </ul>
        <li>Become an Expert in Command Line Python</li>
        </ul>
    </div>
</div>


## Update the exercise!
** !!! Before you do anything, backup your exercises !!! **
#### For Windows: 

Open the Git Shell <img src="files/images/gitshell.png" /> icon (<b>not the blue one</b>). Type in 

<cb>cd cbs-python</cb><br />
<cb>git checkout -f master</cb><br />
<cb>git pull origin master</cb><br /><br  />

#### For MAC and Linux:

Open a terminal. Navigate to the course directory (Whereever you placed it):

<cb>cd ~/Documents/Courses/cbs-python</cb><br/>

Now update the folder using

<cb>git checkout -f master</cb><br />
<cb>git pull origin master</cb><br /><br  />

## BioPython
#### What is BioPython?

>"The Biopython Project is an international association of developers of freely available Python tools for computational molecular biology. Basically, we just like to program in Python and want to make it as easy as possible to use Python for bioinformatics by creating high-quality, reusable modules and scripts." --
http://biopython.org

#### What does it contain? 

>"A lot..."

<center>
<img src="files/images/BioPython.png" width="800px" />
</center>

#### How to install

>Unfortunately, the new version of anaconda doesn't have BioPython installed by default, so it has to be installed manually.<br><br>
<u>For mac and linux:</u><br \>
Go to the terminal and type:<br><b>pip install biopython</b><br>
<br><u>For windows:</u><br>
Find the <i>anaconda command prompt</i> (search for it in windows, or find the program folder, and locate the program), and type the following:<br>
<b>conda install biopython</b><br>


## A few Examples
The best way to explain how to use BioPython is by examples

#### The Alphabet
The BioPython module contains alphabets to declare a sequence type such as DNA and Proteins.

In [ ]:
from Bio import Alphabet
print Alphabet.ThreeLetterProtein.letters

In [ ]:
from Bio.Alphabet import IUPAC
print IUPAC.IUPACProtein.letters
print IUPAC.unambiguous_dna.letters

We can also recreate the codon table we made in one of the previous week's exercises

In [ ]:
from Bio import Data
codonTable = Data.CodonTable.standard_dna_table.forward_table
print codonTable["ATG"]
print codonTable

#### Seq Objects
This objects is composed of a sequence of a specific type (alphabet)

In [ ]:
from Bio.Seq import Seq
my_gene = Seq("CCGGGTT", IUPAC.unambiguous_dna)
my_gene

In [ ]:
my_gene.transcribe()

In [ ]:
my_gene.translate()

Many of the string methods applies on these Seq objects, like

In [ ]:
my_gene[4:]

In [ ]:
len(my_gene)

If you need it to be a string, you can always convert it

In [ ]:
str(my_gene)

**Small exercise**
Try to create a Seq object with some DNA, use help() on the seq object to get some information on what you can do with it (it will be a very long output). Try to figure out how you can reverse complement the sequence

In [ ]:
# Empty cell to write in code for exercise

#### SeqRecord
SeqRecord is a python Class that represents a sequence record containing the sequence itself, name and id. Much like an entry from a fasta file. Let's see how it works

In [ ]:
from Bio.SeqRecord import SeqRecord
my_record = SeqRecord( my_gene , id="001", name="MyGene1",
                      description="My first gene")
print my_record

You can store multiple sequences as a list of SeqRecords: 

In [ ]:
s = Seq("CCGGGTTAGCTAGCTACGTACATCGTACGATC", IUPAC.unambiguous_dna)
my_record2 = SeqRecord( s , id="002", name="MyGene2",
                      description="My second gene")
sequences_list = [my_record, my_record2]
print sequences_list
#print sequences_list[0]

#### SeqIO
This class contains methods for reading and writing sequence files and handle them as SeqRecord objects. This class is extremely useful!

In [ ]:
from Bio import SeqIO

**Reading Sequence files**

If there is only one sequence use ```SeqIO.read()```:

In [ ]:
hbg = SeqIO.read( "../data/human_beta_globin.fasta" , "fasta" )

print hbg

If there is more than one sequence use ```SeqIO.parse()``` (this is the real use of SeqIO). Try to first look at the file (less data/HIV-1_M-B.fasta) - it contains multiple proteins from HIV.

In [ ]:
for record in SeqIO.parse( "../data/HIV-1_M-B.fasta" , "fasta" ):
    print record.id, "- length:", len(record.seq)
    

**Writing to files**

Writing works in the opposite way, turning one or more SeqRecord objects into a file. 

In [ ]:
SeqIO.write( my_record , "../data/my_gene.gbk" , "genbank" )

In [ ]:
SeqIO.read( "../data/my_gene.gbk" , "genbank" )

We can also use it to save multiple sequences (the sequences_list) that we made above, also we can decide which format to save it in (open files and see):

In [ ]:
SeqIO.write(sequences_list, "../data/homemade_sequences.gbk", "genbank")
SeqIO.write(sequences_list, "../data/homemade_sequences.fasta", "fasta")

**Reading a genome in to memory**
Sometimes it is nice to be able to load the entire genome of an organism (say human/bacteria) into python and work with it instead of parsing over it per chromosome/contig. For that we can use the ```SeqIO.to_dict``` function. Small question: Why is the keys/values not in 001->002 ordering?

In [ ]:
sequences_dict = SeqIO.to_dict(SeqIO.parse("../data/homemade_sequences.fasta", "fasta"))
sequences_dict.keys()
#sequences_dict.values()

**Small exercise**
Try to get bases 4-7 in the 001 sequence. Additionally try to get it printed as a string (bases only)

In [ ]:
# Empty cell to write in code for exercise

#### SeqIO.convert
You will be suprised of how much time a bioinformatician spends on getting data converted into the right format. Here ```SeqIO.convert``` will be a great help. Lets try to convert an alignment of 16S rRNA in fasta to eg. phylip format. Open the phylip file and see. You can see all the conversions that are possible here http://biopython.org/wiki/SeqIO#File_Formats

In [ ]:
SeqIO.convert("../data/alignment.fasta", "fasta", "../data/alignment.phylip", "phylip")

### BLAST
The Blast module in BioPython contain functions to deal with blast programs and output. 

** Running a local NCBI BLAST on Interaction**

In [ ]:
from Bio.Blast import NCBIStandalone as BLAST

This module simply works as a Blast wrapper (a piece of code to parse along the commandline options) for the blast program locally installed on the computer. 

In [ ]:
blast_exe = '/usr/cbs/bio/bin/Linux/ia64/blastall'
file_in = '/path/to/my/file.fasta'
blast_db = '/path/to/my/blast/database'

result_handle, error_handle = BLAST.blastall(blast_exe, "blastn", blast_db, file_in)

```result_handle``` contains the blast object from the run. Any errors will be placed in the ```error_handle```.

** Running BLAST over the internet **

The function ```qblast()``` in the ```NCBIWWW``` module calls the *online* version of BLAST. This is slower than the standalone and does not offer the same flexibility for custom databases.

In [ ]:
from Bio.Blast import NCBIWWW

The function takes three arguments: ```qblast(program, database, sequence)```. Here is an example where we just give the Gene ID as sequence:

In [ ]:
result_handle = NCBIWWW.qblast("blastn", "nr", "8332116")

The result of the ```qblast()``` function is similar to a file handle object when we use ```open("myfile")```. Similarly we can use ```read()``` to get the content. The output is nothing more than an XML file.

BioPython offers a tool to process the XML file in the ```NCBIXML``` module:

In [ ]:
from Bio.Blast import NCBIXML

The module has a ```parse()``` that takes as argument the file handle for the blast result and returns an *iterator* for each record in the result. There is *one* record per *query sequence*. In our case, we only had one query sequence, and therefore only one record.

In [ ]:
for record in NCBIXML.parse(result_handle):
    pass


Inside the record object is the *alignments* as well as all the parameters used in the run. Each alignment object contains information of the hit (title, accession number) and a list of *high scoring segment pairs* (HSPS) which is a segment of an alignment, shown below.

<center>
<img src="files/images/blasthsp.png" width="600px" />
</center>

In [ ]:
for align in record.alignments:
    print align.title[:35] + ", E =", align.hsps[0].expect,\
    ", Accession:", str(align.accession)

Let us have a look at the best alignment

In [ ]:
align = record.alignments[0]
for hsp in align.hsps:
    if hsp.expect < 0.01:
        print '****Alignment****'
        print 'sequence:', align.title
        print 'length:', align.length
        print 'e value:', hsp.expect
        print hsp.query[0:75] + '...'
        print hsp.match[0:75] + '...'
        print hsp.sbjct[0:75] + '...'

#### Entrez
**Entrez** is a data retrieval system that provides users access to NCBI’s databases such as PubMed, GenBank, GEO, and many others. You can access Entrez from a web browser to manually enter queries, or you can use Biopython’s Bio.Entrez module for programmatic access to Entrez. The latter allows you for example to search PubMed or download GenBank records from **within a Python script**.


In [ ]:
from Bio import Entrez

Let's see an example of searching for a gene http://www.ncbi.nlm.nih.gov/nucleotide/186972394 directly from the Gene ID: ```"EU490707"```

In [ ]:
entrez_handle = Entrez.efetch(db="nucleotide", id="EU490707",
                              rettype="fasta", email="simon@cbs.dtu.dk")

In [ ]:
entrez_handle.read()

In [ ]:
entrez_handle = Entrez.efetch(db="nucleotide", id="EU490707",
                            rettype="fasta", email="simon@cbs.dtu.dk")

In [ ]:
record = SeqIO.read(entrez_handle, "fasta")

In [ ]:
print record

Before using BioPython to access online resources note that most systems will find it abusing if you make hundreds of requests. BioPython already limits no more than one request every 3 seconds. 

#### Conclusion

There are hundreds of modules and perhaps thousands of functions inside BioPython that you can explore on your own. 

<br>

<center>
<img src="files/images/BioPython.png" width="600px" />
</center>

<br>

A good place to start is the BioPython Tutorial and Cookbook:<br/><br/> 

<center>
http://biopython.org/DIST/docs/tutorial/Tutorial.pdf
</center>

## Become an Expert in Command Line Python

Here are some guidelines of how you should write Biological programs: 

#### Accept parameters to the program
Try not to *hardcode* any parameters to your program. Users of your program may not have enough programming experience to know how to change them inside the code. 

Instead, let them have default values, but accept them to be changed through the command line, like

#### Generate output that can easily be parsed by programs
Always try to find a good balance between output that is human readable and easily parsed by other programs. *Always* use standard formats, like fasta for sequences and csv for a table of values.

Use # in case you need to comment your output. Most other programs will understand this. 

#### Document your code! 
Although this is not specific to be an expert in command line, it is always good practice to keep your code commented and your functions documented. Also, add a "-h" option to run the program, e.g.:

#### Making command line work in Windows

In windows, you cannot readily type in

    python myprogram.py
    
in the command prompt to run ```myprogram.py```. This is because windows doesn't know where Python is. 

To tell it, we have to add the Python directory to something called Environment Variables. 

<ol>
    <li>Open up the run command by holding down <kbd><img id="wds" src="files/images/wds.png" height=10px></kbd> while pressing <kbd>R</kbd>.</li>
    <li>In the field, type <cb>sysdm.cpl</cb> and hit Enter.</li>
    <li>Go to the *Advanced* tab and click on *Environment Variables*. A new dialog window will open.</li>
    <li>Find in the System variables list the varible *Path*: select it and click on Edit.</li>
    <li>(Be careful now) In **the end of the line**, add <cb>;C:\Python27\</cb> and click OK. **Remember the ;**</li>
</ol>

<center>
<img src="files/images/environvar.png" />
</center>
<br />

Now you should be able to open a command prompt and type in

    python
    
regardless of which directory you are in. 

#### Let's try to make our first program using command line inputs!

We can use the sys module to get input from the user.

We'll try to make a program that can take a fasta file as input, and then print the id and length of the sequence, just like we did in the previous example.

Copy the following code into an empty file called "commandline.py" in this notebook directory and try to run it in the terminal by writing:
>commandline.py ../data/alignment.fasta

In [ ]:
#Copy this code into a text editor, and run it on the command-line
import sys
from Bio import SeqIO

for record in SeqIO.parse( sys.argv[1] , "fasta" ):
    print record.id, "- length:", len(record.seq)

As you may have noticed, the sys.argv[1] is the path to the fasta file. What do you think sys.argv[0] is? What happens if you add multiple files in the commandline?

Using sys.argv is fine for quick programs, but if we want to document our code with a help-message like the above example, it can be a bit of a hassle to check every input for mistakes.<br>
Luckily, Python has a module that is made for parsing commandline inputs! It's called argparse. Let's try to solve the previous exercise with argparse

In [ ]:
#Copy this code into a text editor, and run it on the command-line
from argparse import ArgumentParser
from Bio import SeqIO

parser = ArgumentParser(description='Description of program')
parser.add_argument("-f", type=str, dest="fasta_file", default="",
                    help="Fasta input file", required=True)
parser.add_argument("-l", type=int, dest="min_length", default=3,
                    help="Minimum length of sequence to be printed")
#parser.add_argument("-v", dest="verbose", action="store_true",
#                    help="Prints verbose output")

args = parser.parse_args()

for record in SeqIO.parse( args.fasta_file , "fasta" ):
    if len(record.seq) > args.min_length:
        print record.id, "- length:", len(record.seq)

Try to type:
><i>commandline.py ../data/alignment.fasta</i>

Also try to type:
><i>commandline.py -h</i>

Argparse is really useful if you want to avoid hardcoding paths and files to your programs.

In [ ]:
from IPython.core.display import HTML


def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()